In [31]:
import numpy as np
import sib_ldsc_z as ld
from scipy.optimize import minimize
from scipy.special import comb
from scipy.misc import derivative
import scipy.stats
from importlib import reload
import matplotlib.pyplot as plt
reload(ld)

<module 'sib_ldsc_z' from 'c:\\Users\\Hariharan\\Documents\\git_repos\\SNIPar\\ldsc_reg\\inferz\\sib_ldsc_z.py'>

# Defining the PDF and the Log Likelihoods

The likelihood for a SNP $i$ is:

$$
l_i = -\frac{d}{2} log (2 \pi) - \frac{1}{2} log ( |I + r_i S_i^{-1/2} V S_i^{-1/2}| ) - \frac{1}{2} z_i^T (I + r_i S_i^{-1/2} V S_i^{-1/2}) ^{-1} z_i
$$

And its derivative:

$$
\frac{dl}{dV} = r_i S^{-1/2} \Sigma_i^{-1} (\Sigma - z_i z_i^T) \Sigma_i^{-1} S^{-1/2}
$$

In [32]:
np.random.seed(123)

N = int(100)
S_size = int(N/2)
S = np.array([np.array([[.5, 0], [0, .8]]),
    np.array([[0.5, 0], [0, 0.8]])] * S_size )
V = np.identity(2) * 0.5
f = np.random.uniform(0, 1, N)

# N = 100
# S = np.array([0.5/N] * N).reshape((N, 1, 1))
# V = np.atleast_2d(0.5)

In [33]:
model = ld.sibreg(S = S/N)
model.simdata(V/N, N, simr = True)

Warning there is no value for z. Maybe consider simulating it
No value for U given. Generating a vector of ones (all SNPs weighted equally)
No value for r given. Generating a vector of ones for r
Simulated LD scores!
Effect Vectors Simulated!


In [34]:
Vin = hp.extract_upper_triangle(V)
model.neg_logll_grad(Vin)

-496.0564261560453, [[0.5 0. ]
 [0.  0.5]]


(496.0564261560453, array([-8.49013519,  0.14234813, -3.42182671]))

In [35]:
# Testing derivatives
aderiv = model._grad_ll_v(V, model.z[0, :], model.S[0],  model.r[0])

nderiv = model._num_grad_V(V, model.z[0, :], model.S[0],  model.r[0])

np.allclose(aderiv, nderiv)

True

In [36]:
# solving
output, result = model.solve() #), gradfunc = model._num_grad_V
print(result)

No initial guess provided.
Making 'optimal' matrix
-502.24395925596014, [[1.e-06 0.e+00]
 [0.e+00 1.e-06]]
-492.9856082190629, [[ 0.93088205 -0.0151067 ]
 [-0.0151067   0.36501093]]
-448.8474053398347, [[ 6.97560268 -0.11441377]
 [-0.11441377  4.81857516]]
-425.82364735288974, [[12.13580839 -0.20544997]
 [-0.20544997 10.53499634]]
-409.0286699438453, [[18.83266094 -0.36744339]
 [-0.36744339 18.5449561 ]]
-400.27015521289843, [[25.74246425 -0.59823765]
 [-0.59823765 26.7716278 ]]
-396.1121629658576, [[32.66517261 -0.91613092]
 [-0.91613092 34.75261823]]
-394.6805828985441, [[38.38584461 -1.28804962]
 [-1.28804962 40.93900707]]
-394.36967019072137, [[42.05224117 -1.632089  ]
 [-1.632089   44.42949577]]
-394.33820747364905, [[43.59701197 -1.83659738]
 [-1.83659738 45.52377153]]
-394.33694264288215, [[43.97046807 -1.89676211]
 [-1.89676211 45.60085837]]
-394.3368774676067, [[44.0366052  -1.90428749]
 [-1.90428749 45.54864464]]
-394.3368706896527, [[44.04461265 -1.90219298]
 [-1.90219298 45

In [37]:
output

array([[44.04326986, -1.90143063],
       [-1.90143063, 45.51115051]])

In [38]:
modll = model._log_ll(V, model.z[0, :], model.S[0], model.r[0])

In [39]:
V_norm = V
S_inv_root = ld.calc_inv_root(model.S[0])
dist = scipy.stats.multivariate_normal(mean = None,
                                      cov = np.eye(V.shape[0]) + model.r[0] * S_inv_root @ V_norm @ S_inv_root)

nlogll = dist.logpdf(model.z[0, :])
print(nlogll)

-7.331931129444859


In [40]:
modll

-7.331931129444859

# Playing with Derivatives

In [31]:
def alogll_grad(V, S=model.S):
    
    d = S[0].shape[0]
    log_ll = 0
    Gvec = np.zeros((d, d))
    N = len(S)
    V_norm = V/N
    
    for i in range(N):
        Si = S[i]
        zi = model.z[i, :].reshape((d, 1))
        ui = model.u[i]
        ri = model.r[i]
        Si = N * Si 


        log_ll += (1/ui) * model._log_ll(V_norm , zi, Si, ui, ri)
        Gvec += (1/ui) * model._grad_ll_v(V_norm, zi, Si, ui, ri)
        
    return -log_ll, -Gvec/N

In [32]:
def nderiv(V):
    
    g = np.zeros(V.shape)
    for i in range(0,V.shape[0]):
        for j in range(0,V.shape[1]):
            dV = np.zeros((V.shape))
            dV[i,j] = 10 ** (-6)
            V_upper = V+dV
            V_lower = V-dV
            g[i,j] = (alogll_grad(V_upper)[0] - \
                      alogll_grad(V_lower)[0]) / (2 * 10 ** (-6))
    return g


In [33]:
nderiv(V)

array([[-8.49013526,  0.14234803],
       [ 0.14234806, -3.42182679]])

In [34]:
alogll_grad(V)[1]

array([[-8.49013519,  0.14234813],
       [ 0.14234813, -3.42182671]])

In [35]:
np.allclose(alogll_grad(V)[1], nderiv(V))

True

In [36]:
alogll_grad(V)[0]

array([[496.05642616]])

In [42]:
alogll_grad(V, S)

(array([[502.24397211]]), array([[-9.23445231e-10,  1.49860300e-11],
        [ 1.49860300e-11, -3.62094233e-10]]))